In [ ]:
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz

In [19]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

In [20]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
--2022-05-23 04:34:21--  https://jdbc.postgresql.or

In [35]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [22]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [23]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

2643619
2643241
2643241


In [24]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [26]:
df1 = df.drop("marketplace") \
        .drop("product_category") \
        .drop("verified_purchase") \
        .drop("review_headline") \
        .drop("review_body")
df1.show(5)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   11232277|R1001KC9GI6ARJ|B0080YGXPE|     551772997|AETERTEK AT-918C ...|          4|            1|          1|   N| 2015-04-16|
|   18802434|R100B4FHILXAWG|B007TRREJC|      56313707|   Happy Pet Harness|          2|            0|          0|   N| 2014-10-19|
|    5584111|R1017Z5WWPSSLA|B0006342AU|     735846008|SmartyKat Organic...|          5|            0|          0|   N| 2015-02-02|
|   32939702|R1019WUJBM9K3X|B005JW5VRG|     534045437|Marina Naturals F...|          5|            0|          0|   N| 2015-01-20|
|   50738127|R101IZCRPIN8SX|B0029PY7SK|     383930100|ThunderShirt Clas...|        

In [27]:
df1.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



In [61]:
from pyspark.sql.functions import col

In [76]:
from pyspark.sql.functions import to_date

df2=df1.withColumn('review_date', to_date('review_date', 'yyyy-MM-dd'))
df2

DataFrame[customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, review_date: date]

In [73]:
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [78]:
df3=df2.withColumn("total_votes",df2.total_votes.cast('integer')) \
  .withColumn("customer_id",df2.customer_id.cast('integer')) \
  .withColumn("product_parent",df2.product_parent.cast('integer')) \
  .withColumn("star_rating",df2.star_rating.cast('integer')) \
  .withColumn("helpful_votes",df2.helpful_votes.cast('integer'))


In [79]:
df3.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [89]:
df4 = df3.groupBy("customer_id").count()
customers=df4.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   20584931|             8|
|   26074411|            11|
|   16457323|             1|
|   48335968|            16|
|   12508224|             3|
|   37187827|             2|
|   12625518|             3|
|   15983503|             1|
|   16967430|             1|
|    7415586|             3|
|   41176672|             4|
|    2353707|             1|
|   46222570|             3|
|   36868650|             7|
|   45932425|             2|
|   15377634|            10|
|   37508113|             1|
|   46449100|             5|
|   21851331|             1|
|   43833436|            10|
+-----------+--------------+
only showing top 20 rows



In [88]:
review_id_table = df3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001KC9GI6ARJ|   11232277|B0080YGXPE|     551772997| 2015-04-16|
|R100B4FHILXAWG|   18802434|B007TRREJC|      56313707| 2014-10-19|
|R1017Z5WWPSSLA|    5584111|B0006342AU|     735846008| 2015-02-02|
|R1019WUJBM9K3X|   32939702|B005JW5VRG|     534045437| 2015-01-20|
|R101IZCRPIN8SX|   50738127|B0029PY7SK|     383930100| 2014-12-03|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [92]:
products = df3.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0080YGXPE|AETERTEK AT-918C ...|
|B007TRREJC|   Happy Pet Harness|
|B0006342AU|SmartyKat Organic...|
|B005JW5VRG|Marina Naturals F...|
|B0029PY7SK|ThunderShirt Clas...|
+----------+--------------------+
only showing top 5 rows



In [91]:
vine_table = df3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1001KC9GI6ARJ|          4|            1|          1|   N|
|R100B4FHILXAWG|          2|            0|          0|   N|
|R1017Z5WWPSSLA|          5|            0|          0|   N|
|R1019WUJBM9K3X|          5|            0|          0|   N|
|R101IZCRPIN8SX|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"postgres",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)